# coco.data内容（修改）

In [ ]:
'''
parse_data_config()的解析代码来看，可以通过“#”來注释行内容
为什么路径有“..”？因为作者原来的coco数据集是放在和程序同一个父目录下的，所以..跳出程序的目录，在父目录进入数据集文件
'''
classes=80
train=../coco/trainvalno5k.txt  #存放的图片路径
valid=../coco/5k.txt            #测试集图片路径
names=data/coco.names           #coco类别名
backup=backup/
eval=coco

# parse_data_config()解析coco.data

In [ ]:
{'gpus': '0,1,2,3', 
 'num_workers': '10', 
 'classes': '80', 
 'train': '../coco/trainvalno5k.txt', 
 'valid': '../coco/5k.txt', 
 'names': 'data/coco.names', 
 'backup': 'backup/', 
 'eval': 'coco'}

# dataloader的剖析

In [ ]:
'''
dataset.py文件内，类似图片的加载，这里也使用了一个类class load_images_and_labels()，比较复杂。
这里值展示一些中间变量，主要参见github的注释
为了展示方便，我把检测图片换成了自己标注的30张小轿车图片，以下均如此
'''

class load_images_and_labels():  
    def __init__(self, path, batch_size=1, img_size=608, multi_scale=False, augment=False):
        self.path = path
        with open(path, 'r') as file:
            self.img_files = file.readlines()

        self.img_files = [path.replace('\n', '') for path in self.img_files]
        '''
        ['/py/pic/JPEGImages/1.jpg', 
        '/py/pic/JPEGImages/2.jpg', 
        '/py/pic/JPEGImages/3.jpg', 
            ...
        '/py/pic/JPEGImages/30.jpg']不要留空行，否则也会作为元素存进去
        '''
        self.label_files = [path.replace('images', 'labels').replace('.png', '.txt').replace('.jpg', '.txt') for path in
                            self.img_files]
        '''
        ['/py/pic/JPEGImages/1.txt', 
        '/py/pic/JPEGImages/2.txt',
            ...
        '/py/pic/JPEGImages/30.txt']
        '''
        self.nF = len(self.img_files)  # number of image files
        self.nB = math.ceil(self.nF / batch_size)  # number of batches
        self.batch_size = batch_size
        self.height = img_size
        self.multi_scale = multi_scale
        self.augment = augment

        assert self.nB > 0, 'No images found in path %s' % path

        # RGB normalization values
        # self.rgb_mean = np.array([60.134, 49.697, 40.746], dtype=np.float32).reshape((1, 3, 1, 1))
        # self.rgb_std = np.array([29.99, 24.498, 22.046], dtype=np.float32).reshape((1, 3, 1, 1))

    def __iter__(self):
        self.count = -1
        self.shuffled_vector = np.random.permutation(self.nF) if self.augment else np.arange(self.nF)
        '''
        if  not  self.augment:
        [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29]
        if  self.augment:
        [ 2 28 13 10 26 24 27 11 17 22  5 16  8 14 23 20  1 29  6  4 18 19  9  7 25  3  0 21 15 12]
        '''
        return self

    def __next__(self):
        self.count += 1
        if self.count == self.nB:
            raise StopIteration

        ia = self.count * self.batch_size
        ib = min((self.count + 1) * self.batch_size, self.nF)

        if self.multi_scale:
            # Multi-Scale YOLO Training
            height = random.choice(range(10, 20)) * 32  # 320 - 608 pixels
        else:
            # Fixed-Scale YOLO Training
            height = self.height

        img_all = []
        labels_all = []
        for index, files_index in enumerate(range(ia, ib)):
            img_path = self.img_files[self.shuffled_vector[files_index]]
            label_path = self.label_files[self.shuffled_vector[files_index]]

            img = cv2.imread(img_path)  # BGR
            if img is None:
                continue

            augment_hsv = True
            if self.augment and augment_hsv:
                # SV augmentation by 50%
                fraction = 0.50
                img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
                S = img_hsv[:, :, 1].astype(np.float32)
                V = img_hsv[:, :, 2].astype(np.float32)

                a = (random.random() * 2 - 1) * fraction + 1
                S *= a
                if a > 1:
                    np.clip(S, a_min=0, a_max=255, out=S)

                a = (random.random() * 2 - 1) * fraction + 1
                V *= a
                if a > 1:
                    np.clip(V, a_min=0, a_max=255, out=V)

                img_hsv[:, :, 1] = S.astype(np.uint8)
                img_hsv[:, :, 2] = V.astype(np.uint8)
                cv2.cvtColor(img_hsv, cv2.COLOR_HSV2BGR, dst=img)

            h, w, _ = img.shape
            img, ratio, padw, padh = resize_square(img, height=height, color=(127.5, 127.5, 127.5))

            # Load labels
            if os.path.isfile(label_path):
                labels0 = np.loadtxt(label_path, dtype=np.float32).reshape(-1, 5)

                # Normalized xywh to pixel xyxy format
                labels = labels0.copy()
                labels[:, 1] = ratio * w * (labels0[:, 1] - labels0[:, 3] / 2) + padw
                labels[:, 2] = ratio * h * (labels0[:, 2] - labels0[:, 4] / 2) + padh
                labels[:, 3] = ratio * w * (labels0[:, 1] + labels0[:, 3] / 2) + padw
                labels[:, 4] = ratio * h * (labels0[:, 2] + labels0[:, 4] / 2) + padh
            else:
                labels = np.array([])

            # Augment image and labels
            if self.augment:
                img, labels, M = random_affine(img, labels, degrees=(-5, 5), translate=(0.10, 0.10), scale=(0.90, 1.10))

            plotFlag = False
            if plotFlag:
                import matplotlib.pyplot as plt
                plt.figure(figsize=(10, 10)) if index == 0 else None
                plt.subplot(4, 4, index + 1).imshow(img[:, :, ::-1])
                plt.plot(labels[:, [1, 3, 3, 1, 1]].T, labels[:, [2, 2, 4, 4, 2]].T, '.-')
                plt.axis('off')

            nL = len(labels)
            if nL > 0:
                # convert xyxy to xywh
                labels[:, 1:5] = xyxy2xywh(labels[:, 1:5].copy()) / height

            if self.augment:
                # random left-right flip
                lr_flip = True
                if lr_flip & (random.random() > 0.5):
                    img = np.fliplr(img)
                    if nL > 0:
                        labels[:, 1] = 1 - labels[:, 1]

                # random up-down flip
                ud_flip = False
                if ud_flip & (random.random() > 0.5):
                    img = np.flipud(img)
                    if nL > 0:
                        labels[:, 2] = 1 - labels[:, 2]
            #对一张图片遍历结束，列表存入图片信息矩阵和一个包含这张图所有gt的张量，看下面的全部打印输出即可知
            img_all.append(img)
            labels_all.append(torch.from_numpy(labels))

        # Normalize
        img_all = np.stack(img_all)[:, :, :, ::-1].transpose(0, 3, 1, 2)  # BGR to RGB and cv2 to pytorch
        img_all = np.ascontiguousarray(img_all, dtype=np.float32)
        # img_all -= self.rgb_mean
        # img_all /= self.rgb_std
        img_all /= 255.0

        return torch.from_numpy(img_all), labels_all
        '''
        看看返回全部图片的信息：
        首先是torch.from_numpy(img_all)的张量维度：torch.Size([30, 3, 416, 416])，30张图片较好理解
        然后是label:
        [tensor([[0.00000, 0.67837, 0.73789, 0.05795, 0.04066],
                 [0.00000, 0.83071, 0.70716, 0.06600, 0.03726]]), 
         tensor([[0.00000, 0.90200, 0.44104, 0.05873, 0.04905],
                 [0.00000, 0.87276, 0.46533, 0.06384, 0.05178],
                 [0.00000, 0.76982, 0.52693, 0.05216, 0.05285]]), 
         ...
         tensor([[0.00000, 0.88917, 0.21438, 0.07863, 0.03844],
                 [0.00000, 0.91443, 0.32260, 0.08621, 0.04365],
                 [0.00000, 0.90650, 0.36637, 0.10071, 0.04244],
                 [0.00000, 0.73085, 0.45161, 0.05084, 0.06293],
                 [0.00000, 0.75494, 0.53112, 0.05614, 0.05986],
                 [0.00000, 0.78969, 0.61730, 0.06548, 0.06405],
                 [0.00000, 0.68837, 0.35550, 0.09051, 0.05555],
                 [0.00000, 0.46658, 0.41092, 0.09667, 0.03519],
                 [0.00000, 0.14835, 0.43208, 0.10331, 0.05653]])]
          可以看出，每个张量包含一张图片的gt内容，第一张图片有两个gt，信息分别是cxywh（经过416归一化）
          显然，列表长度为30
        '''

    def __len__(self):
        return self.nB  # number of batches



In [ ]:
    for batch_i, (imgs, targets) in enumerate(dataloader):

        with torch.no_grad():   #不追踪梯度
            #通过model(img)自动调用Module类的forward方法计算损失，该步骤放到gpu计算
            output = model(imgs.to(device))     
            output = non_max_suppression(output, conf_thres=conf_thres, nms_thres=nms_thres)

        # Compute average precision for each sample
        for sample_i, (labels, detections) in enumerate(zip(targets, output)):
            correct = []

            if detections is None:
                # If there are no detections but there are labels mask as zero AP
                if labels.size(0) != 0:
                    mAPs.append(0), mR.append(0), mP.append(0)
                continue

            # Get detections sorted by decreasing confidence scores
            detections = detections.cpu().numpy()
            detections = detections[np.argsort(-detections[:, 4])]

            # If no labels add number of detections as incorrect
            if labels.size(0) == 0:
                # correct.extend([0 for _ in range(len(detections))])
                mAPs.append(0), mR.append(0), mP.append(0)
                continue
            else:
                target_cls = labels[:, 0]

                # Extract target boxes as (x1, y1, x2, y2)
                target_boxes = xywh2xyxy(labels[:, 1:5]) * img_size

                detected = []
                for *pred_bbox, conf, obj_conf, obj_pred in detections:

                    pred_bbox = torch.FloatTensor(pred_bbox).view(1, -1)
                    # Compute iou with target boxes
                    iou = bbox_iou(pred_bbox, target_boxes)
                    # Extract index of largest overlap
                    best_i = np.argmax(iou)
                    # If overlap exceeds threshold and classification is correct mark as correct
                    if iou[best_i] > iou_thres and obj_pred == labels[best_i, 0] and best_i not in detected:
                        correct.append(1)
                        detected.append(best_i)
                    else:
                        correct.append(0)

            # Compute Average Precision (AP) per class
            AP, AP_class, R, P = ap_per_class(tp=correct, conf=detections[:, 4], pred_cls=detections[:, 6],
                                              target_cls=target_cls)

            # Accumulate AP per class
            AP_accum_count += np.bincount(AP_class, minlength=nC)
            AP_accum += np.bincount(AP_class, minlength=nC, weights=AP)

            # Compute mean AP across all classes in this image, and append to image list
            mAPs.append(AP.mean())
            mR.append(R.mean())
            mP.append(P.mean())

            # Means of all images
            mean_mAP = np.mean(mAPs)
            mean_R = np.mean(mR)
            mean_P = np.mean(mP)

            # Print image mAP and running mean mAP
            print(('%11s%11s' + '%11.3g' * 3) % (len(mAPs), dataloader.nF, mean_P, mean_R, mean_mAP))

# train和test的dataloader分析：for batch_i, (imgs, targets) in enumerate(dataloader)

In [ ]:


for batch_i, (imgs, targets) in enumerate(dataloader):
        '''
        imgs维度是张量：torch.Size([2, 3, 416, 416])，第一维度是batch_size数我这里设置的2
        targets是列表：如[tensor([[0.00000, 0.67837, 0.73789, 0.05795, 0.04066],
                                [0.00000, 0.83071, 0.70716, 0.06600, 0.03726]]), 
                        tensor([[0.00000, 0.90200, 0.44104, 0.05873, 0.04905],
                                [0.00000, 0.87276, 0.46533, 0.06384, 0.05178],
                                [0.00000, 0.76982, 0.52693, 0.05216, 0.05285]])]
                    含有batch_size个张量的列表，每个张量含有二维数组，第0维是gt个数，第1维是一个gt的信息
        '''
        with torch.no_grad():
            output = model(imgs.to(device))
            '''
            注意：这是在测试集上test效果，实际上也算是detect的一部分，思路和detect的前向传播一样
            前向传播，output列表含2（batch_size）个张量，每个张量维度为torch.Size([10647, 85])
            10647是三尺度合计预测的box数目；85是每个box的信息数目：80+4+1
            '''
            output = non_max_suppression(output, conf_thres=conf_thres, nms_thres=nms_thres)
            '''
            经过NMS，output列表显然还是两个张量元素，不过每个张量维度为torch.Size([31, 7]）因为抑制了部分重叠框
            '''

        # Compute average precision for each sample
        for sample_i, (labels, detections) in enumerate(zip(targets, output)):
            '''
            将预测的输出和gt进行压缩
            re：这个zip创建迭代对像是真的好用
            zip(targets, output)是一个迭代对像，将目标压缩成元组形式，可通过list进行显示，如下只打印第一个元素zip(targets, output)[0]也就是第一张图片和他的box：
                (tensor([[0.00000, 0.67837, 0.73789, 0.05795, 0.04066],[0.00000, 0.83071, 0.70716, 0.06600, 0.03726]]), 
                 tensor([[ 56.15526, 330.78809,  65.46760, 359.50897,   0.69717,   1.00000,   0.00000],
                         [137.66658, 157.83122, 147.64067, 174.90404,   0.60628,   0.99516,   0.00000],
                         [ 17.28005, 147.81779,  26.88957, 162.79787,   0.54086,   0.97253,   0.00000],
                         ...
                         [120.16389, 180.52190, 149.01424, 209.81590,   0.42977,   0.99791,   7.00000]], device='cuda:0'))
            一个元组包含一张图片的gt张量和预测的box张量组成的信息

            '''
            correct = []

            if detections is None:
                # If there are no detections but there are labels mask as zero AP
                if labels.size(0) != 0:
                    mAPs.append(0), mR.append(0), mP.append(0)
                continue

            # Get detections sorted by decreasing confidence scores
            detections = detections.cpu().numpy()
            '''
            对于一张图，这里的nms后生存下来的box信息维度，也就是detections维度为（31,7）的numpy矩阵
            31是box数目，7是nms后85压缩到7个信息4+confidence+class_prob+class_label并且对于同一个标签内是按照confidence高到低排序的
            [[     56.155      330.79      65.468      359.51     0.69717           1           0]
             [     137.67      157.83      147.64       174.9     0.60628     0.99516           0]
             [      17.28      147.82       26.89       162.8     0.54086     0.97253           0]
              ...
             [     120.16      180.52      149.01      209.82     0.42977     0.99791           7]]

            '''
            detections = detections[np.argsort(-detections[:, 4])]
            '''
            原来box是按照每个类标签内conf降序排列，下面转为所有类统一统计进行conf降序排列
            '''
            # If no labels add number of detections as incorrect
            if labels.size(0) == 0:
                # correct.extend([0 for _ in range(len(detections))])
                mAPs.append(0), mR.append(0), mP.append(0)
                continue
            else:
                target_cls = labels[:, 0]
                '''
                label的gt标注每个张量含有五个元素：c,x,y,w,h,如第一张图片
                tensor([[0.00000, 0.67837, 0.73789, 0.05795, 0.04066],
                        [0.00000, 0.83071, 0.70716, 0.06600, 0.03726]])
                '''
                # Extract target boxes as (x1, y1, x2, y2)
                target_boxes = xywh2xyxy(labels[:, 1:5]) * img_size

                detected = []
                
                '''detections维度还是（31,7）下面对一张图的每个预测的box进行遍历！'''
                for *pred_bbox, conf, obj_conf, obj_pred in detections:
                    '''
                    这个*很6,很方便，*代表传递多个参数,会根据你遍历输入进行自动计算个数
                    打印这四个变量看看输出：
                    [4.786209, 313.84634, 53.00174, 353.85886]  0.99444515  0.99444515  0.839418  2.0
                    可以看出7维数据刨去后面四个，剩下的直接整个作为列表传递给pred_bbox了
                    '''
                    
                    pred_bbox = torch.FloatTensor(pred_bbox).view(1, -1)
                    '''转为张量，并加了一维tensor([[  4.78621, 313.84634,  53.00174, 353.85886]])'''
                    
                    # Compute iou with target boxes
                    iou = bbox_iou(pred_bbox, target_boxes)
                    '''
                    先看看输入：
                    pred_bbox：      tensor([[  4.78621, 313.84634,  53.00174, 353.85886]]) （必然只有一个，因为是遍历）
                    target_boxes:    tensor([[270.14755, 298.50504, 294.25320, 315.41931],   （有几个gt就是几维）
                                             [331.84720, 286.42920, 359.30310, 301.92773]])
                    输出：            tensor([0., 0.])      (我自己乱选的图片，iou为0正常)
                    输出是：每个gt的target_boxes（如这里有两个）都要与所有预测出的（遍历的这个）box进行iou计算，输出得到和该图片gt中box数目相同长度的一维张量
                    '''
                    
                    # Extract index of largest overlap
                    best_i = np.argmax(iou)   '''取出最大iou的索引，常数：tensor(0)，也就是找出这个box应该属于哪个gt'''
                    # If overlap exceeds threshold and classification is correct mark as correct
                    if iou[best_i] > iou_thres and obj_pred == labels[best_i, 0] and best_i not in detected:
                        correct.append(1)
                        detected.append(best_i)
                    else:
                        correct.append(0)
            '''所有预测的box遍历完毕，计算该张图片的ap'''
            # Compute Average Precision (AP) per class
            '''AP计算放在下个cell'''
            AP, AP_class, R, P = ap_per_class(tp=correct, conf=detections[:, 4], pred_cls=detections[:, 6],
                                              target_cls=target_cls)

            # Accumulate AP per class
            AP_accum_count += np.bincount(AP_class, minlength=nC)
            AP_accum += np.bincount(AP_class, minlength=nC, weights=AP)

            # Compute mean AP across all classes in this image, and append to image list
            mAPs.append(AP.mean())
            mR.append(R.mean())
            mP.append(P.mean())

            # Means of all images
            mean_mAP = np.mean(mAPs)
            mean_R = np.mean(mR)
            mean_P = np.mean(mP)

            # Print image mAP and running mean mAP
            print(('%11s%11s' + '%11.3g' * 3) % (len(mAPs), dataloader.nF, mean_P, mean_R, mean_mAP))


# AP计算

In [ ]:
'''紧接上文，仍然在一张图片内'''

def ap_per_class(tp, conf, pred_cls, target_cls):
    # lists/pytorch to numpy
    tp, conf, pred_cls, target_cls = np.array(tp), np.array(conf), np.array(pred_cls), np.array(target_cls)
    '''
    先看看这几个输入是什么：
    tp：  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
          长度31,是NMS后剩下的31个预测box，通过iou和寻最大，找出每个box应该属于哪个gt，然后比较conf阈值和label，如果预测正确，加个1,不正确加个0，上面31个0说明预测全错误
    
    conf: [ 0.99445  0.99407   0.98948   0.97971   0.96331   0.91007  0.8762   0.84589  0.81772   0.81035   0.80608   0.80236 
           0.76731  0.70668   0.69717   0.62326   0.62144   0.60628  0.59539  0.55832  0.54974   0.54384   0.54086   0.50405   
           0.48564  0.42977   0.41794   0.40113   0.37924   0.37901  0.36003]
          长度31,每个box按conf从大到小排列  
    
    pred_cls:[ 2 2 2 7 2 2 2 2 7 2 2 2 2 2 0 7 2 0 2 2 2 7 0 7 7 7 2 0 2 2 2]
          对应box预测的label
          
    target_cls:[0 0]   gt的真实label    
    '''
    # Sort by objectness
    i = np.argsort(-conf)
    tp, conf, pred_cls = tp[i], conf[i], pred_cls[i]

    # Find unique classes
    '''
    上面分析过pred_cls, target_cls均为一维向量，通过np.concatenate在第0维融合，相当于把整个target_cls列表添加到pred_cl后面
    然后剔除重复的标签，留下唯一的class标签的行向量
    '''
    unique_classes = np.unique(np.concatenate((pred_cls, target_cls), 0))

    # Create Precision-Recall curve and compute AP for each class
    ap, p, r = [], [], []
    for c in unique_classes:
        i = pred_cls == c
        '''（atten）为什么这里看似布尔量可以数值运算？因为是numpy.ndarray，而不是布尔，矩阵比较后可以计算'''
        n_gt = sum(target_cls == c)  # Number of ground truth objects
        n_p = sum(i)  # Number of predicted objects
        '''
        atten：这里是计算每类的AP，所以按类遍历
        unique_classes=[0 2 7]
        第一次循环c=0,寻找第一个类别:
            i = pred_cls == c将预测的向量每个元素和遍历的类c进行比较，一样为真，得到pred_cls同维（31）的布尔向量
                i=[False False False False False False False False False 
                   False False False False False  True False False  True 
                   False False False False  True False False False False  True False False False]；
            sum(target_cls == c)将真实gt类别和当前遍历类进行比较，得到target_cls同维布尔向量，然后求和，表示的是真实gt中有几个是c这个类；
                n_gt=2
            n_p = sum(i)表示的是预测的box中有几个是这个c类
                n_p=4
        '''

        if (n_p == 0) and (n_gt == 0):
            continue
        elif (n_p == 0) or (n_gt == 0):
            ap.append(0)
            r.append(0)
            p.append(0)
        else:
            # Accumulate FPs and TPs
            fpc = np.cumsum(1 - tp[i])
            tpc = np.cumsum(tp[i])

            # Recall
            recall_curve = tpc / (n_gt + 1e-16)
            r.append(tpc[-1] / (n_gt + 1e-16))

            # Precision
            precision_curve = tpc / (tpc + fpc)
            p.append(tpc[-1] / (tpc[-1] + fpc[-1]))

            # AP from recall-precision curve
            ap.append(compute_ap(recall_curve, precision_curve))

    return np.array(ap), unique_classes.astype('int32'), np.array(r), np.array(p)